# module name: FuzyMapping Initial.ipynb
- purpose: 
- input: 
    - GS1_FoodBev_2022.csv
    - ITM_DSC_ATTRIBUTES_NEW_parquet.gzip

In [0]:
import pandas as pd, numpy as np, rapidfuzz, os
from rapidfuzz import fuzz, utils

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None) 
pd.set_option("display.max_rows", None)

In [0]:
import os

In [0]:
"""
 ITM_DSC_ATTRIBUTES_NEW_parquet.gzip  is used by syndigo mapping
 GS1 has 'AttributeCode', 'AttributeTitle', 'AttributeDefinition', 'AttributeValueCode',  etc.
"""
gs1 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/GS1_FoodBev_2022.csv') # (135,600, 19)
pim_mart = pd.read_parquet('/dbfs/FileStore/tables/DATA_SCIENCE/ITM_DSC_ATTRIBUTES_NEW_parquet.gzip') # (4,937,400, 34)  

/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
 exec(code_obj, self.user_global_ns, self.user_ns)

In [0]:
pim_mart.columns

Out[5]: Index(['ITM_ID', 'GTIN_NO', 'GTIN_NAM', 'CON_UNT_ITM_DSC', 'DSP_UNT_ITM_DSC',
 'PMY_DPT_DSC', 'REC_DPT_DSC', 'DPT_DSC', 'COM_DSC', 'SUBCOM_DSC',
 'TAG_DSC', 'RCT_DSC', 'ITM_LVL_DSC', 'GRP_LVL_DSC',
 'NET_CONT_EQV_QY_VIP_TO_KRG_QY', 'NET_CONT_EQV_UOM_VIP_TO_KRG_UOM',
 'GTIN_HGT_OVD', 'GTIN_LTH_OVD', 'GTIN_WTH_OVD', 'GRS_WGT_OVD',
 'GTIN_NET_WGT_OVD', 'MIN_DAYS_OF_SHF_LIF_AT_WHS_DAYS', 'TPT_CLS_CD',
 'TPT_CLS_VLU', 'MAX_DAYS_OF_SHF_LIF_AT_WHS_DTB_CET_DAYS',
 'TYP_OF_LBL_CD', 'TYP_OF_LBL_VLU', 'SIZ_CD', 'SIZ_CD_VLU', 'SIZ_QY',
 'IS_EDBL_CD', 'IS_EDBL_VLU', 'VND_ECOM_DSC', 'KRG_OWN_ECOM_DSC'],
 dtype='object')

In [0]:
pim_mart.DPT_DSC.unique()

Out[7]: array(['COMMERCIAL BKY', 'GROC-ALL OTHER', 'PERSONAL CARE',
 'NATURAL FOODS', 'CKY/CRKR/SNK', 'CANDY', 'BEEF', 'BEAUTY',
 'HEALTH', 'MENS WEAR', 'FROZEN GROCERY', 'GROC NON-EDIBLE',
 'PACKAGED DELI', 'HBC NON-EDIBLE', 'MUSIC MKT', 'PET',
 'SCHOOL/OFFICE', 'HOUSEWARES', 'FRESH SEAFOOD', 'FURN/HOME DEC',
 'GM MERCH EXP', 'FLORAL-INDOOR', 'TOYS', 'BAKERY', 'SEASONAL',
 'BABY', 'PACKAGE PRODUCE', 'HOME IMPRVMNT', 'VEGETABLES', 'DAIRY',
 'VIDEO/ELEC/COMP', 'FLORAL-OUTDOOR', 'DOMESTICS', 'SPORTING GOODS',
 'POULTRY', 'GARDEN-OUTDOOR', 'REFRIG GROCERY', 'FRESH FOOD DEST',
 'SOFT DRINKS', 'DELI BULK', 'PKG MEAT OTHER', 'NF NON-EDIBLE',
 None, 'PKG MEAT', 'PHONES', 'PKG SEAFOOD', 'STORAGE',
 'SLD SND CMS', 'PKG SPECIALTY', 'FRUIT', 'SPECIALTY CHS',
 'COFFEE SHOP', 'FLORAL WRAP', 'PORK', 'SPIRITS', 'CARDS', 'RX-OTC',
 'ACCESSORIES', 'CHILDRENS', 'SPECIALTY MEAT', 'WINE',
 'READY-TO-WEAR', 'BEER', 'GM-MISC', 'DELI WRAP SPLY',
 'MEAT/SEAF WRAP', 'TOBACCO', 'FRESH PROCESSED', 'INTIMATES',
 'MEAT/SFD STORE', 'DELI STOR SPLY', 'STORE SUPPLY', 'SHOES',
 'MISC SALES TRAN', 'PHARMACY', 'PROD STOR SPLY', 'PROD WRAP SPLY',
 'END OF ORD COUP', 'BAGS FRT END', 'KIOSK GAS', 'BOTTLE DEP/RET',
 'PHARM SUPPLY', 'CNTRL STR SPLY', '<OBSOLETE> DFLT 159 VALUE',
 '<OBSOLETE> FOOD SERVICE', 'WATCHES', 'PHARMACY WRAP',
 'MRKTPLACE SPLY', 'GROCERY SUPPLY', 'SEAFOOD WRAP',
 'PHARM COUPONS', 'COSMETIC SUPPLY', 'OPT DELI WRAP',
 'VIDEO RENTAL', 'MISC FASH JWLRY', 'DIAMONDS'], dtype=object)

In [0]:
df1= gs1

In [0]:
df1.columns

Out[11]: Index(['SegmentCode', 'SegmentTitle', 'SegmentDefinition', 'FamilyCode',
 'FamilyTitle', 'FamilyDefinition', 'ClassCode', 'ClassTitle',
 'ClassDefinition', 'BrickCode', 'BrickTitle',
 'BrickDefinition_Includes', 'BrickDefinition_Excludes', 'AttributeCode',
 'AttributeTitle', 'AttributeDefinition', 'AttributeValueCode',
 'AttributeValueTitle', 'AttributeValueDefinition'],
 dtype='object')

In [0]:
df1.head()

Out[13]:

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition
0,50000000,Food/Beverage/Tobacco,NaN,50200000,Beverages,NaN,50202200,Alcoholic Beverages (Includes De-Alcoholised V...,NaN,10008042,Alcohol Flavouring Kit,Includes any products that can be described/ob...,Excludes products such as Alcohol Making Acces...,NaN,NaN,NaN,NaN,NaN,NaN
1,50000000,Food/Beverage/Tobacco,NaN,50200000,Beverages,NaN,50202200,Alcoholic Beverages (Includes De-Alcoholised V...,NaN,10000142,Alcohol Making Kits,Includes any products that can be described/ob...,Excludes products such as Alcohol Making Acces...,20000004.0,Type of Alcohol Making Kit,"Indicates, with reference to the product brand...",30000309.0,BEER KIT,NaN
2,50000000,Food/Beverage/Tobacco,NaN,50200000,Beverages,NaN,50202200,Alcoholic Beverages (Includes De-Alcoholised V...,NaN,10000142,Alcohol Making Kits,Includes any products that can be described/ob...,Excludes products such as Alcohol Making Acces...,20000004.0,Type of Alcohol Making Kit,"Indicates, with reference to the product brand...",30003009.0,LIQUEUR KIT,NaN
3,50000000,Food/Beverage/Tobacco,NaN,50200000,Beverages,NaN,50202200,Alcoholic Beverages (Includes De-Alcoholised V...,NaN,10000142,Alcohol Making Kits,Includes any products that can be described/ob...,Excludes products such as Alcohol Making Acces...,20000004.0,Type of Alcohol Making Kit,"Indicates, with reference to the product brand...",30002832.0,SPIRIT KIT,NaN
4,50000000,Food/Beverage/Tobacco,NaN,50200000,Beverages,NaN,50202200,Alcoholic Beverages (Includes De-Alcoholised V...,NaN,10000142,Alcohol Making Kits,Includes any products that can be described/ob...,Excludes products such as Alcohol Making Acces...,20000004.0,Type of Alcohol Making Kit,"Indicates, with reference to the product brand...",30002515.0,UNCLASSIFIED,This term is used to describe those product at...


In [0]:
pimmart_grocery = pim_mart[pim_mart.PMY_DPT_DSC == 'GROCERY'] #  (603,733, 34)
pimart_commercial_bakery = pim_mart[pim_mart.DPT_DSC == 'COMMERCIAL BKY'] #(8168,34)

In [0]:
pimart_commercial_bakery.shape


Out[18]: (8168, 34)

In [0]:
pimart_commercial_bakery.info()  # missing GTIN name

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8168 entries, 0 to 4929080
Data columns (total 34 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 ITM_ID 8168 non-null int64 
 1 GTIN_NO 8168 non-null int64 
 2 GTIN_NAM 4343 non-null object 
 3 CON_UNT_ITM_DSC 2498 non-null object 
 4 DSP_UNT_ITM_DSC 2823 non-null object 
 5 PMY_DPT_DSC 8168 non-null object 
 6 REC_DPT_DSC 8168 non-null object 
 7 DPT_DSC 8168 non-null object 
 8 COM_DSC 8168 non-null object 
 9 SUBCOM_DSC 8168 non-null object 
 10 TAG_DSC 5720 non-null object 
 11 RCT_DSC 4706 non-null object 
 12 ITM_LVL_DSC 4563 non-null object 
 13 GRP_LVL_DSC 2972 non-null object 
 14 NET_CONT_EQV_QY_VIP_TO_KRG_QY 5321 non-null float64
 15 NET_CONT_EQV_UOM_VIP_TO_KRG_UOM 5306 non-null object 
 16 GTIN_HGT_OVD 4725 non-null float64
 17 GTIN_LTH_OVD 4725 non-null float64
 18 GTIN_WTH_OVD 4725 non-null float64
 19 GRS_WGT_OVD 4396 non-null float64
 20 GTIN_NET_WGT_OVD 4263 non-null float64
 21 MIN_DAYS_OF_SHF_LIF_AT_WHS_DAYS 4074 non-null float64
 22 TPT_CLS_CD 3927 non-null object 
 23 TPT_CLS_VLU 3927 non-null object 
 24 MAX_DAYS_OF_SHF_LIF_AT_WHS_DTB_CET_DAYS 3109 non-null float64
 25 TYP_OF_LBL_CD 3335 non-null object 
 26 TYP_OF_LBL_VLU 3335 non-null object 
 27 SIZ_CD 4631 non-null object 
 28 SIZ_CD_VLU 4631 non-null object 
 29 SIZ_QY 4705 non-null float64
 30 IS_EDBL_CD 6029 non-null object 
 31 IS_EDBL_VLU 6029 non-null object 
 32 VND_ECOM_DSC 8168 non-null object 
 33 KRG_OWN_ECOM_DSC 5427 non-null object 
dtypes: float64(9), int64(2), object(23)
memory usage: 2.2+ MB

In [0]:
len(gs1)

Out[19]: 135600

In [0]:
gs1.FamilyTitle.unique()

Out[108]: array(['Beverages', 'Bread/Bakery Products',
 'Cereal/Grain/Pulse Products',
 'Confectionery/Sugar Sweetening Products', 'Fish and Seafood',
 'Food/Beverage/Tobacco Variety Packs', 'Fresh Garnish (Food)',
 'Fruits - Unprepared/Unprocessed (Fresh)',
 'Fruits - Unprepared/Unprocessed (Frozen)',
 'Fruits - Unprepared/Unprocessed (Shelf Stable)',
 'Fruits/Vegetables Fresh & Fresh Cut',
 'Fruits/Vegetables Fresh Cut',
 'Fruits/Vegetables/Nuts/Seeds Prepared/Processed',
 'Leaf Vegetables - Unprepared/Unprocessed (Fresh)',
 'Meat/Fish/Seafood Substitutes', 'Meat/Poultry/Other Animals',
 'Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitutes',
 'Nuts/Seeds - Unprepared/Unprocessed (In Shell)',
 'Nuts/Seeds - Unprepared/Unprocessed (Perishable)',
 'Oils/Fats Edible', 'Prepared/Preserved Foods',
 'Seasonings/Preservatives/Extracts',
 'Tobacco/Cannabis/Smoking Accessories',
 'Vegetables - Unprepared/Unprocessed (Frozen)',
 'Vegetables - Unprepared/Unprocessed (Shelf Stable)',
 'Vegetables (Non Leaf) - Unprepared/Unprocessed (Fresh)'],
 dtype=object)

In [0]:
gs1[gs1.FamilyTitle == 'Prepared/Preserved Foods'].BrickTitle.unique()

Out[110]: array(['Baby/Infant - Foods/Beverages Variety Packs',
 'Baby/Infant - Formula (Shelf Stable)',
 'Baby/Infant - Specialised Beverages (Shelf Stable)',
 'Baby/Infant - Specialised Foods (Frozen)',
 'Baby/Infant - Specialised Foods (Shelf Stable)',
 'Dairy Based Products / Meals - Not Ready to Eat/Drink (Frozen)',
 'Dairy Based Products / Meals - Not Ready to Eat/Drink (Perishable)',
 'Dairy/Egg Based Products / Meals - Ready to Eat (Perishable)',
 'Dairy/Egg Based Products / Meals - Ready to Eat (Shelf Stable)',
 'Dairy/Egg Based Products / Meals Variety Packs',
 'Egg Based Products / Meals - Not Ready to Eat (Frozen)',
 'Egg Based Products / Meals - Not Ready to Eat (Perishable)',
 'Egg Based Products / Meals - Not Ready to Eat (Shelf Stable)',
 'Dessert Sauces/Toppings/Fillings (Frozen)',
 'Dessert Sauces/Toppings/Fillings (Perishable)',
 'Dessert Sauces/Toppings/Fillings (Shelf Stable)',
 'Desserts (Frozen)', 'Desserts (Perishable)',
 'Desserts (Shelf Stable)',
 'Desserts/Dessert Toppings Variety Packs',
 'Ice Cream/Ice Novelties (Frozen)',
 'Ice Cream/Ice Novelties (Shelf Stable)',
 'Dough Based Products / Meals - Not Ready to Eat - Savoury (Frozen)',
 'Dough Based Products / Meals - Not Ready to Eat - Savoury (Perishable)',
 'Dough Based Products / Meals - Not Ready to Eat - Savoury (Shelf Stable)',
 'Dough Based Products / Meals - Ready to Eat - Savoury (Perishable)',
 'Dough Based Products / Meals - Ready to Eat - Savoury (Shelf Stable)',
 'Dough Based Products / Meals Variety Packs',
 'Grain Based Products / Meals - Not Ready to Eat - Savoury (Frozen)',
 'Grain Based Products / Meals - Not Ready to Eat - Savoury (Perishable)',
 'Grain Based Products / Meals - Not Ready to Eat - Savoury (Shelf Stable)',
 'Grain Based Products / Meals - Ready to Eat - Savoury (Perishable)',
 'Grain Based Products / Meals - Ready to Eat - Savoury (Shelf Stable)',
 'Grain Based Products / Meals Variety Packs',
 'Pasta/Noodles - Not Ready to Eat (Frozen)',
 'Pasta/Noodles - Not Ready to Eat (Perishable)',
 'Pasta/Noodles - Not Ready to Eat (Shelf Stable)',
 'Pasta/Noodles - Ready to Eat (Perishable)',
 'Pasta/Noodles - Ready to Eat (Shelf Stable)',
 'Pasta/Noodles Variety Packs', 'Soup Additions (Frozen)',
 'Soup Additions (Perishable)', 'Soup Additions (Shelf Stable)',
 'Soups - Prepared (Frozen)', 'Soups - Prepared (Perishable)',
 'Soups - Prepared (Shelf Stable)',
 'Soups - Prepared Variety Packs',
 'Prepared/Preserved Foods Variety Packs',
 'Ready-Made Combination Meals - Not Ready to Eat (Frozen)',
 'Ready-Made Combination Meals - Not Ready to Eat (Perishable)',
 'Ready-Made Combination Meals - Not Ready to Eat (Shelf Stable)',
 'Ready-Made Combination Meals - Not Ready to Eat Variety Packs',
 'Ready-Made Combination Meals - Ready to Eat (Perishable)',
 'Ready-Made Combination Meals - Ready to Eat (Shelf Stable)',
 'Ready-Made Combination Meals - Ready to Eat Variety Packs',
 'Sandwiches/Filled Rolls/Wraps (Frozen)',
 'Sandwiches/Filled Rolls/Wraps (Perishable)',
 'Sandwiches/Filled Rolls/Wraps Variety Packs',
 'Caramel/Toffee Apples',
 'Chips/Crisps/Snack Mixes - Natural/Extruded (Shelf Stable)',
 'Doodles/ Puffs', 'Popcorn (Shelf Stable)',
 'Salt Sticks / Mini Pretzels', 'Snacks Other',
 'Snacks Variety Packs',
 'Confectionery Based Spreads (Shelf Stable)',
 'Honey (Shelf Stable)', 'Honey Substitutes',
 'Jams/Marmalades (Shelf Stable)',
 'Jams/Marmalades/Fruit Spreads (Perishable)',
 'Sweet Spreads Variety Packs',
 'Vegetable Based Products / Meals - Not Ready to Eat (Frozen)',
 'Vegetable Based Products / Meals - Not Ready to Eat (Perishable)',
 'Vegetable Based Products / Meals - Not Ready to Eat (Shelf Stable)',
 'Vegetable Based Products / Meals - Ready to Eat (Perishable)',
 'Vegetable Based Products / Meals - Ready to Eat (Shelf Stable)',
 'Vegetable Based Products / Meals Variety Packs'], dtype=object)

In [0]:
gs1.columns

Out[69]: Index(['SegmentCode', 'SegmentTitle', 'SegmentDefinition', 'FamilyCode',
 'FamilyTitle', 'FamilyDefinition', 'ClassCode', 'ClassTitle',
 'ClassDefinition', 'BrickCode', 'BrickTitle',
 'BrickDefinition_Includes', 'BrickDefinition_Excludes', 'AttributeCode',
 'AttributeTitle', 'AttributeDefinition', 'AttributeValueCode',
 'AttributeValueTitle', 'AttributeValueDefinition'],
 dtype='object')

### Fuzzy Matching each PIMMART DEPT to GS1 Class/Classes

In [0]:
pimart_commercial_bakery.ITM_ID.nunique()

Out[81]: 8168

In [0]:
pimmart_grocery.DPT_DSC.size

Out[27]: 603733

In [0]:
pimmart_groacery.DPT_DSC  #pandas.core.series.Series; size = 603.733

Out[70]: 0 COMMERCIAL BKY
1 GROC-ALL OTHER
2 COMMERCIAL BKY
3 COMMERCIAL BKY
5 GROC-ALL OTHER
 ... 
4937358 GROC-ALL OTHER
4937359 GROC-ALL OTHER
4937369 GROC-ALL OTHER
4937370 GROC-ALL OTHER
4937382 GROC-ALL OTHER
Name: DPT_DSC, Length: 603733, dtype: object

In [0]:
pim_match = pd.DataFrame(pimmart_grocery.groupby('DPT_DSC').apply(lambda x: ' '.join(x.COM_DSC.unique())),columns = ['COM_DSC_text']).reset_index() #DF (16,2)

In [0]:
pim_match.head()

Out[23]:

,DPT_DSC,COM_DSC_text
0,BEER,CRAFT/MICRO BEERS SINGLES BEER OTHER CIDER IMP...
1,BOTTLE DEP/RET,BOTTLE DEPOSITS BOTTLE REFUNDS MISCELLANEOUS T...
2,CKY/CRKR/SNK,BAG SNACKS SS/VENDING - SALTY SNACKS WAREHOUSE...
3,COMMERCIAL BKY,BAKED BREADS BAKED SWEET GOODS MISCELLANEOUS T...
4,DAIRY,YOGURT MILK BY-PRODUCTS REFRGRATD JUICES/DRINK...


In [0]:
(pim_match.DPT_DSC + ' ' + pim_match.COM_DSC_text)

Out[15]: 0 BEER CRAFT/MICRO BEERS SINGLES BEER OTHER CIDE...
1 BOTTLE DEP/RET BOTTLE DEPOSITS BOTTLE REFUNDS ...
2 CKY/CRKR/SNK BAG SNACKS SS/VENDING - SALTY SNA...
3 COMMERCIAL BKY BAKED BREADS BAKED SWEET GOODS ...
4 DAIRY YOGURT MILK BY-PRODUCTS REFRGRATD JUICES...
5 END OF ORD COUP MISCELLANEOUS TRANSACTIONS
6 FROZEN GROCERY FROZEN DESSERTS FROZEN AUTHENTI...
7 GROC NON-EDIBLE LAUNDRY DETERGENTS FACIAL TISS...
8 GROC-ALL OTHER COLD CEREAL SPICES & EXTRACTS F...
9 NATURAL FOODS NF CRACKERS NF FROZEN NOVELTIES ...
10 NF NON-EDIBLE NF LAUNDRY NF HOUSEHOLD NF DISH ...
11 PET PET CARE SUPPLIES WET CAT FOOD DRY DOG FOO...
12 REFRIG GROCERY DRY CHEESE CHEESE BUTTER/MARGAR...
13 SOFT DRINKS SPECIALTY SOFT DRINKS SOFT DRINKS ...
14 SPIRITS TEQUILA RTD COCKTAILS N. AMER WHISKEY ...
15 WINE TABLE WINE MISC WINE SPARKLING WINE MISCE...
dtype: object

In [0]:
pim_match['PIMMART_Matching_Text'] = (pim_match.DPT_DSC + ' ' + pim_match.COM_DSC_text).str.lower()

In [0]:
pim_match.to_csv('PIMMART_DEP_COM_matchingtext_051723.csv', index = None)  # where is the location of the csv file 

In [0]:
gs1['FamilyClassTitle'] = gs1.FamilyTitle + '/'+ gs1.ClassTitle
gs1_match =  pd.DataFrame(gs1.groupby('FamilyClassTitle').apply(lambda x: ' '.join(x.BrickTitle.unique())),columns = ['Class_Brick_text']).reset_index()
gs1_match['GS1_Matching_Text'] = (gs1_match.FamilyClassTitle + ' ' + gs1_match.Class_Brick_text).str.lower()

In [0]:
gs1_match


Out[19]:

,FamilyClassTitle,Class_Brick_text,GS1_Matching_Text
0,Beverages/Alcoholic Beverages (Includes De-Alc...,Alcohol Flavouring Kit Alcohol Making Kits Alc...,beverages/alcoholic beverages (includes de-alc...
1,Beverages/Beverages Variety Packs,Beverages Variety Packs,beverages/beverages variety packs beverages va...
2,Beverages/Coffee/Coffee Substitutes,Coffee - Capsules/Pods Coffee - Ground Beans C...,beverages/coffee/coffee substitutes coffee - c...
3,Beverages/Coffee/Tea/Substitutes,Coffee/Tea/Substitutes Variety Packs,beverages/coffee/tea/substitutes coffee/tea/su...
4,Beverages/Non Alcoholic Beverages - Not Ready ...,Chocolate/Cocoa/Malt - Not Ready to Drink Dair...,beverages/non alcoholic beverages - not ready ...
...,...,...,...
133,Vegetables (Non Leaf) - Unprepared/Unprocessed...,Agretti Glasswort Sea Kale Sea Lavender Tidal ...,vegetables (non leaf) - unprepared/unprocessed...
134,Vegetables (Non Leaf) - Unprepared/Unprocessed...,Cherry Tomatoes - Oblong Cherry Tomatoes - Rou...,vegetables (non leaf) - unprepared/unprocessed...
135,Vegetables (Non Leaf) - Unprepared/Unprocessed...,Vegetables - Unprepared/Unprocessed (Fresh) Va...,vegetables (non leaf) - unprepared/unprocessed...
136,Vegetables - Unprepared/Unprocessed (Frozen)/V...,Vegetables - Unprepared/Unprocessed (Frozen),vegetables - unprepared/unprocessed (frozen)/v...


In [0]:
pim_match.shape

Out[29]: (16, 3)

In [0]:
pim_match

Out[20]:

,DPT_DSC,COM_DSC_text,PIMMART_Matching_Text
0,BEER,CRAFT/MICRO BEERS SINGLES BEER OTHER CIDER IMP...,beer craft/micro beers singles beer other cide...
1,BOTTLE DEP/RET,BOTTLE DEPOSITS BOTTLE REFUNDS MISCELLANEOUS T...,bottle dep/ret bottle deposits bottle refunds ...
2,CKY/CRKR/SNK,BAG SNACKS SS/VENDING - SALTY SNACKS WAREHOUSE...,cky/crkr/snk bag snacks ss/vending - salty sna...
3,COMMERCIAL BKY,BAKED BREADS BAKED SWEET GOODS MISCELLANEOUS T...,commercial bky baked breads baked sweet goods ...
4,DAIRY,YOGURT MILK BY-PRODUCTS REFRGRATD JUICES/DRINK...,dairy yogurt milk by-products refrgratd juices...
5,END OF ORD COUP,MISCELLANEOUS TRANSACTIONS,end of ord coup miscellaneous transactions
6,FROZEN GROCERY,FROZEN DESSERTS FROZEN AUTHENTIC ASIAN ICE CRE...,frozen grocery frozen desserts frozen authenti...
7,GROC NON-EDIBLE,LAUNDRY DETERGENTS FACIAL TISSUE LAUNDRY ADDIT...,groc non-edible laundry detergents facial tiss...
8,GROC-ALL OTHER,COLD CEREAL SPICES & EXTRACTS FRUIT SNACKS CON...,groc-all other cold cereal spices & extracts f...
9,NATURAL FOODS,NF CRACKERS NF FROZEN NOVELTIES NF FROZEN ENTR...,natural foods nf crackers nf frozen novelties ...


In [0]:
from rapidfuzz import fuzz

In [0]:
gs1_match.FamilyClassTitle.iloc[0]

Out[22]: 'Beverages/Alcoholic Beverages (Includes De-Alcoholised Variants)'

In [0]:
def match_gs1_levels_x(pim_ix , gs1_matchlist = list(gs1_match.GS1_Matching_Text), cutoff = 40):
    out = pd.DataFrame(columns= ['KROGER_DEPT','GS1_FAMILY/CLASS','MATCH_SCORE'])
    for i, j  in enumerate(gs1_matchlist):
        score = fuzz.token_set_ratio(pim_match.PIMMART_Matching_Text.iloc[pim_ix],j, processor=utils.default_process)
        if score >=cutoff:
            out.loc[i] = [pim_match.DPT_DSC.iloc[pim_ix],gs1_match.FamilyClassTitle.iloc[i],score]
    return out.sort_values('MATCH_SCORE', ascending = False).reset_index(drop= True)

In [0]:
res_x = pd.DataFrame(columns= ['KROGER_DEPT','GS1_FAMILY/CLASS','MATCH_SCORE'])
for ix in range(len(pim_match)):
    res_x = res_x.append(match_gs1_levels_x(ix, cutoff= 45)).reset_index(drop = True)

In [0]:
def match_gs1_levels(pim_ix , gs1_matchlist = list(gs1_match.GS1_Matching_Text), cutoff = 40):
    out = pd.DataFrame(columns= ['KROGER_DEPT','GS1_FAMILY/CLASS','MATCH_SCORE'])
    for i, j  in enumerate(gs1_matchlist):
        score = fuzz.token_set_ratio(pim_match.PIMMART_Matching_Text.iloc[pim_ix],j)
        if score >=cutoff:
            out.loc[i] = [pim_match.DPT_DSC.iloc[pim_ix],gs1_match.FamilyClassTitle.iloc[i],score]
    return out.sort_values('MATCH_SCORE', ascending = False).reset_index(drop= True)

In [0]:
res = pd.DataFrame(columns= ['KROGER_DEPT','GS1_FAMILY/CLASS','MATCH_SCORE'])
for ix in range(len(pim_match)):
    res = res.append(match_gs1_levels(ix, cutoff= 45)).reset_index(drop = True)
    

In [0]:
res

Out[46]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE
0,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,47.031963
1,CKY/CRKR/SNK,Prepared/Preserved Foods/Snacks,46.831956
2,DAIRY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,48.672566
3,DAIRY,Prepared/Preserved Foods/Dairy/Egg Based Produ...,46.853147
4,DAIRY,Prepared/Preserved Foods/Grain Based Products ...,45.112782
5,FROZEN GROCERY,Fruits/Vegetables Fresh Cut/Vegetables - Fresh...,47.462277
6,GROC NON-EDIBLE,Tobacco/Cannabis/Smoking Accessories/Cannabis ...,45.416667
7,SOFT DRINKS,Beverages/Beverages Variety Packs,46.017699


In [0]:
res_x.rename(columns={'MATCH_SCORE':'MATCH_SCORE_preprocess_yes'})


Out[48]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE_preprocess_yes
0,BEER,Beverages/Beverages Variety Packs,56.250000
1,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,51.562500
2,BEER,Beverages/Non Alcoholic Beverages - Not Ready ...,49.266862
3,BEER,Beverages/Non Alcoholic Beverages - Ready to D...,47.222222
4,BEER,Vegetables (Non Leaf) - Unprepared/Unprocessed...,45.751634
...,...,...,...
177,REFRIG GROCERY,Prepared/Preserved Foods/Dough Based Products ...,45.423729
178,REFRIG GROCERY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,45.323741
179,SOFT DRINKS,Prepared/Preserved Foods/Baby/Infant - Foods/B...,49.411765
180,SOFT DRINKS,Beverages/Tea and Infusions/Tisanes,47.457627


In [0]:
res.rename(columns={'MATCH_SCORE':'MATCH_SCORE_preprocess_none'})

Out[49]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE_preprocess_none
0,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,47.031963
1,CKY/CRKR/SNK,Prepared/Preserved Foods/Snacks,46.831956
2,DAIRY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,48.672566
3,DAIRY,Prepared/Preserved Foods/Dairy/Egg Based Produ...,46.853147
4,DAIRY,Prepared/Preserved Foods/Grain Based Products ...,45.112782
5,FROZEN GROCERY,Fruits/Vegetables Fresh Cut/Vegetables - Fresh...,47.462277
6,GROC NON-EDIBLE,Tobacco/Cannabis/Smoking Accessories/Cannabis ...,45.416667
7,SOFT DRINKS,Beverages/Beverages Variety Packs,46.017699


In [0]:
res_merge=  pd.merge(res, res_x,  how="outer", on=["KROGER_DEPT", "GS1_FAMILY/CLASS"])

In [0]:
res_merge 

Out[58]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE_x,MATCH_SCORE_y
0,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,47.0320,51.5625
1,CKY/CRKR/SNK,Prepared/Preserved Foods/Snacks,46.8320,48.6957
2,DAIRY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,48.6726,60.3015
3,DAIRY,Prepared/Preserved Foods/Dairy/Egg Based Produ...,46.8531,52.6749
4,DAIRY,Prepared/Preserved Foods/Grain Based Products ...,45.1128,46.4730
5,FROZEN GROCERY,Fruits/Vegetables Fresh Cut/Vegetables - Fresh...,47.4623,50.1475
6,GROC NON-EDIBLE,Tobacco/Cannabis/Smoking Accessories/Cannabis ...,45.4167,47.0320
7,SOFT DRINKS,Beverages/Beverages Variety Packs,46.0177,NaN
8,BEER,Beverages/Beverages Variety Packs,NaN,56.2500
9,BEER,Beverages/Non Alcoholic Beverages - Not Ready ...,NaN,49.2669


In [0]:
res_merge.rename(columns = {'match_score':'res_match_score'})  # shape: (8, 3)

Out[43]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE
0,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,47.031963
1,CKY/CRKR/SNK,Prepared/Preserved Foods/Snacks,46.831956
2,DAIRY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,48.672566
3,DAIRY,Prepared/Preserved Foods/Dairy/Egg Based Produ...,46.853147
4,DAIRY,Prepared/Preserved Foods/Grain Based Products ...,45.112782
5,FROZEN GROCERY,Fruits/Vegetables Fresh Cut/Vegetables - Fresh...,47.462277
6,GROC NON-EDIBLE,Tobacco/Cannabis/Smoking Accessories/Cannabis ...,45.416667
7,SOFT DRINKS,Beverages/Beverages Variety Packs,46.017699


In [0]:
res_merge

Out[55]:

,KROGER_DEPT,GS1_FAMILY/CLASS,MATCH_SCORE_x,MATCH_SCORE_y
0,BEER,Beverages/Alcoholic Beverages (Includes De-Alc...,47.031963,51.562500
1,CKY/CRKR/SNK,Prepared/Preserved Foods/Snacks,46.831956,48.695652
2,DAIRY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,48.672566,60.301508
3,DAIRY,Prepared/Preserved Foods/Dairy/Egg Based Produ...,46.853147,52.674897
4,DAIRY,Prepared/Preserved Foods/Grain Based Products ...,45.112782,46.473029
...,...,...,...,...
178,REFRIG GROCERY,Prepared/Preserved Foods/Dough Based Products ...,NaN,45.423729
179,REFRIG GROCERY,Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitu...,NaN,45.323741
180,SOFT DRINKS,Prepared/Preserved Foods/Baby/Infant - Foods/B...,NaN,49.411765
181,SOFT DRINKS,Beverages/Tea and Infusions/Tisanes,NaN,47.457627


In [0]:
res.to_csv('PIMMART_DEPT_TO_GS1_CLASS_LEVEL_MAPPING_062023.csv', index = None)

In [0]:
pim_match_1= pd.DataFrame(pimmart_grocery.groupby('VND_ECOM_DSC').apply(lambda x: ' '.join(x.SUBCOM_DSC.unique())),columns = ['SUBCOM_DSC_text']).reset_index()

In [0]:
pimart_commercial_bakery['ITEM_SUBCOM_DPT'] = (pimart_commercial_bakery.VND_ECOM_DSC + ' ' + pimart_commercial_bakery.SUBCOM_DSC+' ' + pimart_commercial_bakery.DPT_DSC).str.lower()

<command-1424199737389167>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pimart_commercial_bakery['ITEM_SUBCOM_DPT'] = (pimart_commercial_bakery.VND_ECOM_DSC + ' ' + pimart_commercial_bakery.SUBCOM_DSC+' ' + pimart_commercial_bakery.DPT_DSC).str.lower()

In [0]:
pimart_commercial_bakery['ITEM_SUBCOM_DPT']

Out[97]: 0 lewis bake shop texas toast half loaf bread ma...
2 snack cakes, little debbie family pack back to...
3 snack cakes, little debbie family pack back to...
17 lewis bake shop half loaf hearty rye bread mai...
18 snack cakes, little debbie family pack back to...
 ... 
4924051 host donets dbl choc bag donuts - bagged comme...
4924154 host donets crunch bag donuts - bagged commerc...
4926558 krispy kreme doughnuts with m&m's lto donuts -...
4927407 entenmann's chocolate chip crumb loaf cake, 13...
4929080 san luis sourdough jalapeno artisan style brea...
Name: ITEM_SUBCOM_DPT, Length: 8168, dtype: object

In [0]:
gs1['FamilyClassBrickTitle'] = gs1.FamilyTitle + ' '+ gs1.ClassTitle + ' '+ gs1.BrickTitle

In [0]:
gs1_unique_brick= gs1.drop_duplicates('FamilyClassBrickTitle')
gs1_unique_brick['FamilyClassBrickTitle']= gs1_unique_brick['FamilyClassBrickTitle'].str.lower()

<command-1424199737389178>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 gs1_unique_brick['FamilyClassBrickTitle']= gs1_unique_brick['FamilyClassBrickTitle'].str.lower()

In [0]:
gs1_unique_brick['FamilyClassBrickTitle']

Out[92]: 0 beverages alcoholic beverages (includes de-alc...
1 beverages alcoholic beverages (includes de-alc...
7 beverages alcoholic beverages (includes de-alc...
17 beverages alcoholic beverages (includes de-alc...
18 beverages alcoholic beverages (includes de-alc...
 ... 
134693 vegetables (non leaf) - unprepared/unprocessed...
134994 vegetables (non leaf) - unprepared/unprocessed...
135295 vegetables (non leaf) - unprepared/unprocessed...
135596 vegetables (non leaf) - unprepared/unprocessed...
135597 vegetables (non leaf) - unprepared/unprocessed...
Name: FamilyClassBrickTitle, Length: 899, dtype: object

In [0]:
gs1_unique_brick = gs1_unique_brick[gs1_unique_brick.FamilyTitle.isin(['Bread/Bakery Products', 'Prepared/Preserved Foods'])]

In [0]:
gs1_unique_brick


Out[113]:

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition,FamilyClassBrickTitle
13886,50000000,Food/Beverage/Tobacco,NaN,50180000,Bread/Bakery Products,NaN,50181700,Baking/Cooking Mixes/Supplies,NaN,10000155,Baking/Cooking Mixes (Frozen),Includes any products that can be described/ob...,"Excludes products such as Soups, Perishable or...",20000056.0,Diabetic Claim,"Indicates, with reference to the product brand...",30002960.0,NO,NaN,bread/bakery products baking/cooking mixes/sup...
13928,50000000,Food/Beverage/Tobacco,NaN,50180000,Bread/Bakery Products,NaN,50181700,Baking/Cooking Mixes/Supplies,NaN,10000068,Baking/Cooking Mixes (Perishable),Includes any products that can be described/ob...,"Excludes products such as Soups, Frozen or She...",20000056.0,Diabetic Claim,"Indicates, with reference to the product brand...",30002960.0,NO,NaN,bread/bakery products baking/cooking mixes/sup...
13973,50000000,Food/Beverage/Tobacco,NaN,50180000,Bread/Bakery Products,NaN,50181700,Baking/Cooking Mixes/Supplies,NaN,10000156,Baking/Cooking Mixes (Shelf Stable),Includes any products that can be described/ob...,"Excludes products such as Soups, Frozen or Per...",20000056.0,Diabetic Claim,"Indicates, with reference to the product brand...",30002960.0,NO,NaN,bread/bakery products baking/cooking mixes/sup...
14015,50000000,Food/Beverage/Tobacco,NaN,50180000,Bread/Bakery Products,NaN,50181700,Baking/Cooking Mixes/Supplies,NaN,10000595,Baking/Cooking Mixes/Supplies Variety Packs,Includes any products that can be described/ob...,Excludes products such as Cake Mixes with Grou...,NaN,NaN,NaN,NaN,NaN,NaN,bread/bakery products baking/cooking mixes/sup...
14016,50000000,Food/Beverage/Tobacco,NaN,50180000,Bread/Bakery Products,NaN,50181700,Baking/Cooking Mixes/Supplies,NaN,10000157,Baking/Cooking Supplies (Frozen),Includes any products that can be described/ob...,Excludes products such as Perishable and Shelf...,20000175.0,Suitability for Vegetarians/Vegans Claim,"Indicates, with reference to the product brand...",30002960.0,NO,NaN,bread/bakery products baking/cooking mixes/sup...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76808,50000000,Food/Beverage/Tobacco,NaN,50190000,Prepared/Preserved Foods,NaN,50193100,Vegetable Based Products / Meals,NaN,10000290,Vegetable Based Products / Meals - Not Ready t...,Includes any products that can be described/ob...,Excludes Ready-Made Combination Meals. Specifi...,20000232.0,Differentiating Ingredient,"Indicates, with reference to the product brand...",30003106.0,AQUATIC INVERTEBRATE,NaN,prepared/preserved foods vegetable based produ...
76867,50000000,Food/Beverage/Tobacco,NaN,50190000,Prepared/Preserved Foods,NaN,50193100,Vegetable Based Products / Meals,NaN,10000292,Vegetable Based Products / Meals - Not Ready t...,Includes any products that can be described/ob...,Excludes Ready-Made Combination Meals. Specifi...,20000232.0,Differentiating Ingredient,"Indicates, with reference to the product brand...",30003106.0,AQUATIC INVERTEBRATE,NaN,prepared/preserved foods vegetable based produ...
76930,50000000,Food/Beverage/Tobacco,NaN,50190000,Prepared/Preserved Foods,NaN,50193100,Vegetable Based Products / Meals,NaN,10000289,Vegetable Based Products / Meals - Ready to Ea...,Includes any products that can be described/ob...,Excludes Ready-Made Combination Meals. Specifi...,20000232.0,Differentiating Ingredient,"Indicates, with reference to the product brand...",30003106.0,AQUATIC INVERTEBRATE,NaN,prepared/preserved foods vegetable based produ...
76981,50000000,Food/Beverage/Tobacco,NaN,50190000,Prepared/Preserved Foods,NaN,50193100,Vegetable Based Products / Meals,NaN,10000288,Vegetable Based Products / Meals - Ready to Ea...,Includes any produ

In [0]:
def match_gs1_levels_1(pim_ix , gs1_matchlist_1 = list(gs1_unique_brick.FamilyClassBrickTitle), cutoff = 40):
    out = pd.DataFrame(columns= ['KROGER_VND_ECOM_DSC','BRICK','MATCH_SCORE'])
    for i, j  in enumerate(gs1_matchlist_1):
        score = fuzz.token_set_ratio(pimart_commercial_bakery.VND_ECOM_DSC.iloc[pim_ix],j)
        if score >=cutoff:
            out.loc[i] = [pimart_commercial_bakery.VND_ECOM_DSC.iloc[pim_ix],gs1_unique_brick.BrickTitle.iloc[i],score]
    return out.sort_values('MATCH_SCORE', ascending = False).reset_index(drop= True)

In [0]:
res = pd.DataFrame(columns= ['KROGER_VND_ECOM_DSC','BRICK','MATCH_SCORE'])
for ix in range(len(pimart_commercial_bakery)):
    res = res.append(match_gs1_levels_1(ix, cutoff= 40)).reset_index(drop = True)
    

In [0]:
res.head(20)

Out[119]:

,KROGER_VND_ECOM_DSC,BRICK,MATCH_SCORE
0,Lewis Bake Shop Texas Toast Half Loaf Bread,Sweet Bakery Products Variety Packs,41.758242
1,Lewis Bake Shop Half Loaf Hearty Rye Bread,Sweet Bakery Products Variety Packs,42.222222
2,Lewis Bake Shop Half Loaf Hearty Rye Bread,Cakes - Sweet (Frozen),40.860215
3,Lewis Bake Shop Half Loaf Hearty Rye Bread,Cakes - Sweet (Shelf Stable),40.404040
4,Brownberry Country White Sandwich Bread,Bread/Bakery Products Variety Packs,43.243243
5,Brownberry Country White Sandwich Bread,Bread (Shelf Stable),41.975309
6,Brownberry Country White Sandwich Bread,Sandwiches/Filled Rolls/Wraps (Frozen),41.176471
7,Brownberry Country White Sandwich Bread,Bread (Frozen),40.000000
8,Brownberry Whole Grains 12 Grain Sandwich Bread,Sandwiches/Filled Rolls/Wraps (Frozen),41.818182
9,Brownberry Whole Grains 12 Grain Sandwich Bread,Bread/Bakery Products Variety Packs,41.463415


In [0]:
res.to_csv('/dbfs/FileStore/tables/DATA_SCIENCE/COMMERCIAL_BAKERY_BRICK.csv', index = None)